In [ ]:
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

import jax
from dataclasses import dataclass
import jax.numpy as jnp
import jax.random as jr

import tensorflow_probability.substrates.jax.bijectors as tfb

#with install_import_hook("gpjax", "beartype.beartype"):
import gpjax as gpx
from gpjax.distributions import GaussianDistribution
import matplotlib.pyplot as plt
from matplotlib import rcParams
# plt.style.use(
#     "https://raw.githubusercontent.com/JaxGaussianProcesses/GPJax/main/docs/examples/gpjax.mplstyle"
# )
# colors = rcParams["axes.prop_cycle"].by_key()["color"]

key = jr.PRNGKey(123)

import tensorflow_probability.substrates.jax as tfp
tfd = tfp.distributions
from gpjax.kernels.base import AdditiveKernel


import optax as ox
import tensorflow_probability.substrates.jax.bijectors as tfb



from typing import List, Union, Callable
from jaxtyping import Num, Float
from gpjax.typing import Array, ScalarFloat
from beartype.typing import Optional
from gpjax.base import Module, param_field,static_field
import cola
from cola.linalg.decompositions.decompositions import Cholesky
from jax import vmap
from scipy.stats import qmc



100_000 entries sampled across time/lat/lon over first day of data
X2D has:

0"Sea surface temperature (K)"

1"Sensible heat flux (W/m^2)"

2"Latent heat flux (W/m^2)"

3"Vertically-integrated moisture convergence (kg/m^2)"

4"Column relative humidity (%)"

X3D has:

0"Absolute temperature (K)"

1"Relative humidity (%)"

2"Specific humidity (kg/kg)"

3"Geopotential height (m^2 s^-2)"

4"Zonal wind (m/s)"

5"Meridional wind (m/s)"

6"Potential temperature (K)"

7"Equivalent potential temperature (K)"

8"Equivalent potential temperature saturation deficit (K)"

9"Saturated equivalent potential temperature (K)"

10"MSE-conserving plume buoyancy (m/s^2)"

static has:

0"Land-sea mask"

1"Angle of sub-gridscale orography (rad)

2"Anisotropy of sub-gridscale orography"

3"Standard deviation of sub-gridscale orography"

4"Slope of sub-gridscale orography"

Y has:

0"ERA5 Precipitation (mm/hr)"

1"TRMM Precipitation (mm/hr)"

2"TRMM Relative Error (%)"

plev are

1000., 2000., 3000., 5000., 7000., 10000., 15000., 20000., 25000., 30000., 40000., 50000., 60000., 70000.,80000., 85000., 90000., 92500., 95000., 97500., 100000.

In [ ]:
from utils import VerticalDataset, ProblemInfo



def prep_data(type=str):
    #load data
    # X2d_raw = jnp.array(jnp.load("../data/ERA/NPY_DATA/X2d_sample.npy"), dtype=jnp.float64) # [N, D]
    # X3d_raw = jnp.array(jnp.load("../data/ERA/NPY_DATA/X3d_sample.npy"), dtype=jnp.float64) # [N, D]
    # Xstatic_raw = jnp.array(jnp.load("../data/ERA/NPY_DATA/XStatic_sample.npy"), dtype=jnp.float64) # [N, D]
    # Y_raw = jnp.array(jnp.load("../data/ERA/NPY_DATA/Y_sample.npy"), dtype=jnp.float64) # [N, 1]
    X2d_raw = jnp.array(jnp.load("../data/100_000_one_day/X2D_sample.npy"), dtype=jnp.float64) # [N, D]
    X3d_raw = jnp.array(jnp.load("../data/100_000_one_day/X3D_sample.npy"), dtype=jnp.float64) # [N, D]
    Xstatic_raw = jnp.array(jnp.load("../data/100_000_one_day/XSTATIC_sample.npy"), dtype=jnp.float64) # [N, D]
    Y_raw = jnp.array(jnp.load("../data/100_000_one_day/Y_sample.npy"), dtype=jnp.float64) # [N, 1]


    pressure = jnp.array([[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,60000,70000,80000, 85000,90000,92500,95000,97500,100000]], dtype=jnp.float64)
    # random shuffle
    X2d = jr.permutation(key, X2d_raw)
    X3d = jr.permutation(key, X3d_raw)
    Xstatic = jr.permutation(key, Xstatic_raw)
    Y = jr.permutation(key, Y_raw)

    # look at ERA5 rain
    Y = Y[:,0:1] 

    # just keep the "prognostic" 3d inputs that joe considered (for now)
    # RH, tehta_e^+, theta_e, theta_e^*
    names_3d =  ["Absolute temperature,","Relative Humidity", "Specific Humidity", "Geopotential Height", "Zonal Wind","Meridional Wind","Potential Temperature (theta)","Equivalent Potential Temperature (tehta_e)", "Equivalent Potential Temperature Saturation Deficit (theta_e+)", "Saturated Equivalent Potential Temperature (theta_e*)", "MSE-conserving Plume Buoyancy"]
    names_3d_short =  ["K,","RH", "q", "gh", "wind_z","wind_m","theta","tehta_e", "theta_e+", "theta_e*", "plume"]
    idx_3d = [1,4,5,6,7,8,9]
    # idx_3d = [1, 7, 8, 9]
    names_3d = [names_3d[i] for i in idx_3d]
    names_3d_short = [names_3d_short[i] for i in idx_3d]
    X3d = X3d[:,idx_3d,:]

    names_static = ["Land-sea Mask","Angle of sub-gridscale orography","Anisotropy of sub-gridscale orography","Stdev of sub-gridscale orography","Slope of sub-gridscale orography"]
    names_static_short = ["LSM","O_angle","O_anisotrophy","O_sd","O_slope"]
    idx_static = [0, 3]
    names_static = [names_static[i] for i in idx_static]
    names_static_short = [names_static_short[i] for i in idx_static]
    Xstatic = Xstatic[:,idx_static]
    lsm_threshold = 0.5
    # # also use his "normalisatopm" for sigma_o
    o_sd_idx = names_static.index("Stdev of sub-gridscale orography")
    lsm_idx = names_static.index("Land-sea Mask")
    # Xstatic = Xstatic.at[:, o_sd_idx].set(jnp.where(Xstatic[:,lsm_idx]<lsm_threshold, jnp.nanmean(1.0+jnp.log(1+Xstatic[:,o_sd_idx][Xstatic[:,lsm_idx]>lsm_threshold])), 1.0+jnp.log(1+Xstatic[:,o_sd_idx]))) # optimize lsm_threshold?
    Xstatic = Xstatic.at[:, o_sd_idx].set(jnp.log(Xstatic[:,o_sd_idx]+1.0))
    #map = lambda x: jnp.log((x+1e-5)/(1-x+1e-5))
    # Xstatic = Xstatic.at[:,lsm_idx].set(map(Xstatic[:,lsm_idx]))
    # lsm_threshold = map(lsm_threshold)




    # names_2d = ["Sea Surface temperature", "Sensible heat flux land","Sensible heat flux sea", "Latent heat flux land","Latent heat flux sea", "Vertically-integrated moisture convergence", "Column relative humidity"]
    # names_2d_short = ["T_surface","flux_s_land","flux_s_sea","flux_l_land","flux_l_sea","moisture","CRH"]
    names_2d = ["Sea Surface temperature", "Sensible heat flux", "Latent heat flux", "Vertically-integrated moisture convergence", "Column relative humidity"]
    names_2d_short = ["T_surface","flux_s","flux_l","moisture","CRH"]
    idx_2d = [1,2]
    names_2d = [names_2d[i] for i in idx_2d]
    names_2d_short = [names_2d_short[i] for i in idx_2d]
    X2d = X2d[:,idx_2d]
    #sea_surface_idx = names_2d.index("Sea Surface temperature")
    # sea_surface = jnp.where(Xstatic[:,lsm_idx]>lsm_threshold, jnp.nanmean(X2d[:,sea_surface_idx]), X2d[:, sea_surface_idx])
    # X2d = X2d.at[:,sea_surface_idx].set(sea_surface) # turn nans corresonping to land for sea surface to fixed value, they will be ignored by switch kernels anyway 
    flux_s_idx = names_2d.index("Sensible heat flux")
    flux_l_idx = names_2d.index("Latent heat flux")
    #X2d = X2d.at[:,flux_s_idx].set(jnp.log(X2d[:,flux_s_idx]+0.01 - jnp.min(X2d[:,flux_s_idx])) )
    # X2d = X2d.at[:,flux_l_idx].set(jnp.log(X2d[:,flux_l_idx]+1.0 - jnp.min(X2d[:,flux_l_idx])))

    #remove all pressure levels above 500 hPA
    lowest_idx =  7 #11
    print(f"Removed all pressure levels below {pressure[:,lowest_idx]} hPa")
    X3d = X3d[:, :, lowest_idx:]
    pressure_levels = pressure[:,lowest_idx:]
    pressure_mean = jnp.mean(pressure_levels)
    pressure_std = jnp.std(pressure_levels)
    pressure_levels = (pressure_levels - pressure_mean) / pressure_std

    # remove any entries with nan
    X3d_nan_idx = jnp.isnan(X3d).any(axis=1).any(axis=1)
    X2d_nan_idx = jnp.isnan(X2d).any(axis=1) # ignore sea surface
    Xstatic_nan_idx = jnp.isnan(Xstatic).any(axis=1)
    Y_nan_idx = jnp.isnan(Y).any(axis=1)
    any_nan = X3d_nan_idx | X2d_nan_idx | Y_nan_idx | Xstatic_nan_idx
    no_nan = ~ any_nan
    print(f"Removed {any_nan.sum()} entries with nan")
    X2d = X2d[no_nan,:]
    X3d = X3d[no_nan,:,:]
    Xstatic = Xstatic[no_nan,:]
    Y = Y[no_nan,:]



    if type == "no rain":
        # remove no rain days
        print(f"Removed {(Y[:,0]==0).sum()} entries with zero rain")
        X3d = X3d[Y[:,0]>0,:]
        X2d = X2d[Y[:,0]>0,:]
        Xstatic = Xstatic[Y[:,0]>0,:]
        Y = Y[Y[:,0]>0,:]
    elif type == "just rain":
        # convert output to binary 
        Y = jnp.where(Y>0, 1.0, 0.0)
    elif type == "all":
        pass   
    else:
        raise ValueError("type must be 'no rain' or 'just rain'")


    num_2d_variables= X2d.shape[1]
    num_3d_variables= X3d.shape[1]
    num_static_variables= Xstatic.shape[1]
    num_not_3d_variables = num_2d_variables + num_static_variables
    num_variables = num_2d_variables + num_3d_variables + num_static_variables
    print(f"using {num_static_variables} static variables")
    print(f"using {num_2d_variables} 2d variables")
    print(f"using {num_3d_variables} 3d variables")
    names = names_3d + names_2d + names_static
    names_short = names_3d_short + names_2d_short + names_static_short
    print(f"using variables with names {names_short}")

    problem_info = ProblemInfo(
        num_2d_variables = num_2d_variables,
        num_3d_variables = num_3d_variables,
        num_static_variables = num_static_variables,
        names_2d_short = names_2d_short,
        names_3d_short = names_3d_short,
        names_static_short = names_static_short,
        names_2d = names_2d,
        names_3d = names_3d,
        names =names,
        names_short = names_short,
        names_static = names_static,
        num_variables = num_variables,
        pressure_levels = pressure_levels,
        pressure_mean = pressure_mean,
        pressure_std = pressure_std,
        lsm_threshold = lsm_threshold,
    )

    D = VerticalDataset(
        X2d = X2d,
        X3d = X3d,
        Xstatic = Xstatic,
        y=Y,
        standardize=True,
        standardize_with_NF=False,
        problem_info=problem_info,
    )

    return D, problem_info
    

    

In [ ]:
from plotting import plot_data
D, problem_info = prep_data("just rain")
plot_data(problem_info,D)
D = D.get_subset(500)



In [ ]:
from models import ConjugatePrecipGP, VerticalSmoother, init_smoother
from plotting import plot_params
initial_posterior = ConjugatePrecipGP(
    kernel = gpx.kernels.RBF(lengthscale=jnp.array([1.0]*problem_info.num_variables), variance=1.0),
    likelihood = gpx.likelihoods.Gaussian(num_datapoints=D.n),
    smoother = init_smoother(problem_info)
)


opt_initial_posterior, history = gpx.fit_scipy(
        model=initial_posterior,
        objective=jax.jit(initial_posterior.loss_fn(negative=True)), #todo rejit
        train_data=D,
        # optim=ox.adamw(1e-2),
        # num_iters=1000,
        # batch_size=1024,
        safe=False,
        # key=key,
    )

plt.plot(history)
plot_params(problem_info, opt_initial_posterior,D)